In [20]:
import os
import pandas as pd
from datetime import datetime
import pytz

def append_csv_files(folder_path, start_date, end_date):
    # Convert input date strings to date objects
    start_date = datetime.strptime(start_date, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y%m%d")

    # Get all files in the specified folder
    files = os.listdir(folder_path)

    # List to store dataframes
    dataframes = []

    # Timezone conversion setup
    hk_tz = pytz.timezone('Asia/Hong_Kong')

    # Iterate over files in the folder
    for file in files:
        if file.endswith('.csv'):
            # Extract date from filename
            file_date_str = file.split('.')[0]
            file_date = datetime.strptime(file_date_str, "%Y%m%d")

            # Check if file date is within the specified range
            if start_date <= file_date <= end_date:
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                
                # Assign column name to the first column
                df.columns = ['datetime'] + list(df.columns[1:])
                # Convert to Hong Kong time
                df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_convert(hk_tz).dt.tz_localize(None)

                dataframes.append(df)

    # Concatenate all dataframes
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)

        # Sort by datetime
        combined_df.sort_values(by='datetime', inplace=True)

        # Use folder name as the new CSV file name
        folder_name = os.path.basename(os.path.normpath(folder_path))
        output_file = f"{folder_name}.csv"
        combined_df.to_csv(output_file, index=False)

        print(f"Combined CSV file saved as: {output_file}")
    else:
        print("No CSV files found in the specified date range.")


In [22]:
# 指定folder、startdate和enddate
folder_path = "EURUSD"
start_date = '20240101'
end_date = '20240831'
append_csv_files(folder_path, start_date, end_date)

Combined CSV file saved as: EURUSD.csv
